# Load library

In [ ]:
import csv
import pickle
import os
import re
import functools
import collections
import sys


import numpy as np
import matplotlib.pyplot as plt

sys.path.append('C:/Users/Tianle/Documents/cs231n/spring1617/my-scripts')
from dl.utils.gen_conv_params import *

%load_ext autoreload
%autoreload 2

# Process clinical data

In [ ]:
clinicalfiles = ['clinical/' + v for v in os.listdir('clinical') 
                 if not re.search('Dict|Fields|clinical|overlapping', v)]
# read seven .csv files:
clinicalfiles_acronym = ['train', 'sc1_val', 'sc1_train', 'sc2_val', 'sc2_train', 
                         'sc3_val', 'sc3_train']

clinical_info = {}
for i, file in enumerate(clinicalfiles):
    clinical = []
    with open(file) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            clinical.append(row)
    # The first row is feature names
    var_names = clinical[0]
    # Map feature name to idx
    var_names = {name: i for i, name in enumerate(var_names)}
    del clinical[0]
    clinical_info[clinicalfiles_acronym[i]] = (clinical, var_names)

path = 'processed_data/'
fileName = 'clinical_info.pkl'
with open(path + fileName, 'wb') as f:
    pickle.dump(clinical_info, f)

In [ ]:
# Training dataset
clinical, var_names = clinical_info['train']

# Patient ID are unique
assert len([v[1] for v in clinical]) == len(set([v[1] for v in clinical]))

# SamplId are unique if not 'NA'
# This allows using (filename, sampleID) or just sampleID to find patientID
for name in [v for v in var_names if re.search('SamplId', v)]:
    sids = [v[var_names[name]] for v in clinical
            if v[var_names[name]] != 'NA']
    print(name, len(sids))
    assert len(sids) == len(set(sids))

# Find expression files in two folders: MA and RNA-seq
expfiles = [(root, files) for root, subdir, files in os.walk('Expression Data/') if len(files) > 0]
# Select files with entrezID
expfilepaths = [[os.path.join(root, f) for f in files if re.search('entrezID', f)] 
                for root, files in expfiles]
# Save filenames
expfilenames = [[f for f in files if re.search('entrezID', f)] for root, files in expfiles]

In [ ]:
# do not use 'CENSORED' or 'D_PFS' <= 30 (might be error in the data)
clinical = [v for v in clinical if v[var_names['HR_FLAG']] != 'CENSORED' 
       and float(v[var_names['D_PFS']]) > 30]

In [ ]:
# Overlapping entrezIds
entrez_ids = []
with open('clinical/overlappingEntrezIds.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        entrez_ids.append(row)

entrez_ids = [v[0] for i, v in enumerate(entrez_ids) if i > 0]

# Ensure entrez_ids is ordered
idx_entrezIds = np.array(entrez_ids).argsort()
entrez_ids = [entrez_ids[i] for i in idx_entrezIds]

# Prepare RNA-seq gene expression data (only one file)

In [ ]:
exp = []
with open(expfilepaths[1][0]) as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        exp.append(row)

sampleIds = [n for i, n in enumerate(exp[0]) if i > 0]
sampleIds = {sid: i for i, sid in enumerate(sampleIds)}
# (idx_clinical, idx_exp) map samples that have both clinical data and exp data 
idx_map = [(i, sampleIds[v[var_names['RNASeq_geneLevelExpFileSamplId']]]) 
           for i, v in enumerate(clinical) 
           if v[var_names['RNASeq_geneLevelExpFile']] in expfilenames[1] 
           and v[var_names['RNASeq_geneLevelExpFileSamplId']] in sampleIds]
entrezIds = [v[0] for i, v in enumerate(exp) if i > 0]
# sort entrezIds
idx_entrezIds = np.array(entrezIds).argsort()
entrezIds = [entrezIds[i] for i in idx_entrezIds]
exp = [[float(e) if e != 'NA' else 0 for j, e in enumerate(v) if j > 0]
       for i, v in enumerate(exp) if i > 0]
exp = np.array(exp)
exp = exp.T
exp = exp[[idx_exp for idx_clinical, idx_exp in idx_map]]
exp = exp[:, idx_entrezIds]
assert set(entrez_ids) <= set(entrezIds)
idx_entrezIds = [entrezIds.index(v) for v in entrez_ids]
exp = exp[:, idx_entrezIds]
exp_rnaseq = np.log2(exp + 1)

clinical_rnaseq = [clinical[idx_clinical] for idx_clinical, idx_exp in idx_map]
y_rnaseq = [v[var_names['HR_FLAG']] for v in clinical_rnaseq]

path = 'processed_data/'
fileName = 'rnaseq_features_unnormalized.pkl'
with open(path + fileName, 'wb') as f:
    pickle.dump({'exp': exp_rnaseq, 'y': y_rnaseq, 'entrezIds': entrez_ids, 
                 'clinical': clinical_rnaseq, 'var_names': var_names}, f)

# Prepare microarray gene expression data

In [ ]:
microarray = {}
for i, expfile in enumerate(expfilepaths[0]):
    exp = []
    with open(expfile) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            exp.append(row)
    # First row are sample ids, and first column is entrezIDs 
    sampleIds = [n for i, n in enumerate(exp[0]) if i > 0]
    sampleIds = {sid: i for i, sid in enumerate(sampleIds)}
    # (idx_clinical, idx_exp) map samples that have both clinical data and exp data 
    idx_map = [(i, sampleIds[v[var_names['MA_geneLevelExpFileSamplId']]]) 
               for i, v in enumerate(clinical) 
               if v[var_names['MA_geneLevelExpFile']] in expfilenames[0] 
               and v[var_names['MA_geneLevelExpFileSamplId']] in sampleIds]
    entrezIds = [v[0] for i, v in enumerate(exp) if i > 0]
    # sort entrezIds
    idx_entrezIds = np.array(entrezIds).argsort()
    entrezIds = [entrezIds[i] for i in idx_entrezIds]
    # There are very few 'NA' values. Set them to 0
    exp = [[float(e) if e != 'NA' else 0 for j, e in enumerate(v) if j > 0] 
           for i, v in enumerate(exp) if i > 0]
    exp = np.array(exp)
    exp = exp.T
    exp = exp[[idx_exp for idx_clinical, idx_exp in idx_map]]
    exp = exp[:, idx_entrezIds]
    assert set(entrez_ids) <= set(entrezIds)
    idx_entrezIds = [entrezIds.index(v) for v in entrez_ids]
    exp = exp[:, idx_entrezIds]
    print(expfile, len(sampleIds), exp.shape)
    microarray[expfilenames[0][i]] = {'exp': exp, 'idx_clinical': [i for i, j in idx_map], 
                                      'entrezIds': entrez_ids}

In [ ]:
ma_exp = []
idx_clinical = []
for k, v in microarray.items():
    ma_exp.append(v['exp'])
    idx_clinical = idx_clinical + v['idx_clinical']

exp_ma = np.concatenate(ma_exp, axis=0)

clinical_ma = [clinical[i] for i in idx_clinical]

y_ma = [v[var_names['HR_FLAG']] for v in clinical_ma]
path = 'processed_data/'
fileName = 'ma_features_unnormalized.pkl'
with open(path + fileName, 'wb') as f:
    pickle.dump({'exp': exp_ma, 'y': y_ma, 'entrezIds': entrez_ids, 
                 'clinical': clinical_ma, 'var_names': var_names}, f)

# Preprocess reactome pathway data

In [ ]:
path = 'F:/KnowledgeBases/processedData/'
filename = path + 'ncbi2reactome.csv'
gene_pathway = []
with open(filename) as csvfile:
    reader = csv.reader(csvfile)
    for v in reader:
        gene_pathway.append(v)
gene_to_pathway = collections.defaultdict(list)
pathway_to_gene = collections.defaultdict(list)
for entrezID, pathwayID in gene_pathway:
    gene_to_pathway[pathwayID].append(entrezID)
    pathway_to_gene[entrezID].append(pathwayID)

In [ ]:
def gene_filter_cor_std(exp, clinical, entrezIds, filter_below=0.10, eps=1e-8):
    assert isinstance(exp, np.ndarray)
    # First filter out genes with std <= eps
    std = np.std(exp, axis=0)
    idx = np.where(std > eps)[0]
    std = std[idx]
    exp = exp[:, idx]
    entrezIds = [e for i, e in enumerate(entrezIds) if i in idx]
    y_tmp = np.array([[float(v[var_names['D_PFS']]), float(v[var_names['D_OS']]),
                     float(v[var_names['D_OS_FLAG']]), float(v[var_names['D_PFS_FLAG']]),
                     1 if v[var_names['HR_FLAG']] == 'TRUE' else 0] for v in clinical])
    cor = np.corrcoef(np.concatenate([exp, y_tmp], axis=1), rowvar=False)
    # corrcoef between y and entrezIds
    cor = cor[:exp.shape[1], -5:]
    # based on intuition
    cor_std = np.mean(np.abs(cor) * std[:, None], axis=1)
    # filter out entrezIds that have a small cor_std value
    idx = np.where(cor_std - min(cor_std) > (max(cor_std) - min(cor_std)) * filter_below)[0]
    exp = exp[:, idx]
    entrezIds = [e for i, e in enumerate(entrezIds) if i in idx]
    return exp, entrezIds

_, genes1 = gene_filter_cor_std(exp_rnaseq, clinical_rnaseq, entrez_ids)
_, genes2 = gene_filter_cor_std(exp_ma, clinical_ma, entrez_ids)

genes = sorted(set(genes1).intersection(genes2))

In [ ]:
idx_to_gene = {e: [i] for i, e in enumerate(genes)}
res = reduce_projections([idx_to_gene, gene_to_pathway, pathway_to_gene, 
                         gene_to_pathway, pathway_to_gene, gene_to_pathway, 
                         pathway_to_gene, gene_to_pathway, pathway_to_gene])
num_elements = [len(v) for v in res[0]]
print('number of elements from the first layer to last layer:\n{0}'
      .format(num_elements))

k = len(num_elements) - 2
for i in range(len(num_elements) - 1, 3, -1):
    if (num_elements[i], num_elements[i-1]) == (num_elements[i-2], num_elements[i-3]):
        k = k - 1
    else:
        break
if k < len(num_elements) - 2:
    print('From {0}th layer, repeat'.format(k))
    for i in range(k, len(num_elements)-2):
        assert res[0][i] == res[0][i + 2]
        assert res[1][i] == res[1][i + 2]
    # k-1 and k+1 layers are almost the same
    assert res[0][k-1].keys() == res[0][k+1].keys()
    c = 0
    for p, v in res[0][k-1].items():
        v1 = set([res[1][k-1][e] for e in v])
        v2 = set([res[1][k+1][e] for e in res[0][k+1][p]])
        assert v1 <= v2
        if v1 < v2:
            c += 1
    print('{0} genes/pathways connect to less pathways/genes in layer {1} (res[0][{1}])'
          .format(c, k-1))
else:
    print('No repeat, check the input again!')

idx = [entrez_ids.index(g) for g in res[1][1]]
exp_ma = exp_ma[:, idx]
exp_rnaseq = exp_rnaseq[:, idx]
entrez_ids = res[1][1]

In [ ]:
path = 'processed_data/'
filename = 'exp_features_unnormalized.pkl'
with open(path+filename, 'wb') as f:
    pickle.dump({'exp_ma': exp_ma, 'exp_rnaseq': exp_rnaseq, 
                 'clinical_ma': clinical_ma, 'clinical_rnaseq': clinical_rnaseq,
                 'entrezIds': entrez_ids, 'var_names': var_names, 'res': res}, f)